In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM, Dense,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention, Dropout
from cashare.day_data import daily_data
from cashare.stock_list import stock_list
from cashare.minute_data import m_data

result = pd.DataFrame(columns=['percent','name','ts_code'])
stocks=stock_list(type='us',token='f9882e4b76023e40f506c3b274ecde2295t1')
print(stocks)
for stock in stocks.values:
    try:
        data = m_data(sk_code=stock[0],token='f9882e4b76023e40f506c3b274ecde2295t1',type='1min',start_date='2023-06-20',end_date='2023-07-05')
    except Exception as e:
        continue
    print(data)
    if(data.shape[0]<500):
        continue
    data = data[['close', 'volume']]
    data['volume'] = pd.to_numeric(data['volume'],errors='coerce')
    data.sort_index(ascending=False, inplace=True)
    data = data.reset_index(drop=True)

    # 计算差值
    data_diff = data.diff().dropna()
    # print(data_diff)

    # 数据缩放
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_diff)

    lookback = 50

    # 定义生成器函数
    def generator(data, lookback=10, delay=1, batch_size=20):
        max_index = len(data) - delay - 1
        i = lookback
        while 1:
            if i + delay >= max_index:
                i = lookback
            samples = np.zeros((batch_size, lookback, data.shape[-1]))
            targets = np.zeros((batch_size, data.shape[-1]))  # targets now is 2D, predict price and volume
            for j in range(batch_size):
                if i + delay >= max_index:
                    i = lookback
                rows = np.arange(i - lookback, i)
                samples[j] = data[rows]
                targets[j] = data[i + delay]  # target is the next value sequence
                i += 1
            print(samples)
            print(targets)
            yield samples, targets

    ##lstm
    inputs = Input(shape=(lookback, data.shape[-1]))
    x = LSTM(100, return_sequences=True)(inputs)  # first LSTM layer
    x = Dropout(0.5)(x)
    x = LSTM(100, return_sequences=False)(x)  # second LSTM layer
    outputs = Dense(data.shape[-1])(x)   # only predict the next value
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(), loss='mse')

    train_data = data_scaled[:int(len(data_scaled)*0.9)]
    val_data = list(data_scaled[int(len(data_scaled)*0.1):])

    
    
    last_sequence = np.reshape(train_data[-lookback:], (1, lookback, data_diff.shape[-1]))
    next_prediction = model.predict(last_sequence, verbose=0)[0, 0]
    next_real = val_data[0][0]
    prediction_increase = next_prediction > 0
    real_increase = next_real > 0
    correct = 0
    if prediction_increase == real_increase:
        correct = 1
    new_result = pd.DataFrame([[correct,stock[0],stock[1]]], columns=result.columns)
    print(new_result)
    new_result.to_csv('result_us.csv', mode='a', header=False, index=False)
    
    # #预测涨跌
    # correct_predictions = 0
    # total_predictions = 0 

    # for epoch in range(30):  # train for 30 epochs
    #     train_gen = generator(train_data, lookback=lookback)
    #     model.fit(train_gen, epochs=1, steps_per_epoch=20, verbose=0)  # train the model for one epoch

    #     # move the first data point from val_data to train_data
    #     train_data = np.concatenate([train_data, np.reshape(val_data.pop(0), (1, -1))])

    #     # 预测下一个点
    #     last_sequence = np.reshape(train_data[-lookback:], (1, lookback, data_diff.shape[-1]))
    #     next_prediction = model.predict(last_sequence, verbose=0)[0, 0]

    #     # 计算预测值与真实值之间的差距
    #     if len(val_data) > 0:
    #         next_real = val_data[0][0]  # the next real value
    #         error = np.abs((next_prediction - next_real) / next_real)  # the relative error
    #         print(f'Epoch {epoch + 1}, validation error: {error:.4f}')

    #         # 对预测结果进行涨跌预测
    #         prediction_increase = next_prediction > 0
    #         real_increase = next_real > 0
    #         if prediction_increase == real_increase:
    #             correct_predictions += 1
    #         total_predictions += 1

    # # 计算预测准确率
    # accuracy = correct_predictions / total_predictions
    # print(f'Prediction accuracy: {accuracy:.4f}')
    # new_result = pd.DataFrame([[accuracy,stock[0],stock[1]]], columns=result.columns)
    # print(new_result)
    # result.append(new_result)
    # new_result.to_csv('result_us.csv', mode='a', header=False, index=False)
    



      symbol                             name   price  \
0          A       Agilent Technologies, Inc.  119.32   
1         AA                Alcoa Corporation   34.27   
2        AAC     Ares Acquisition Corporation   10.56   
3     AAC-UN     Ares Acquisition Corporation   10.71   
4     AAC-WT  Ares Acquisition Corporation Re  0.8375   
...      ...                              ...     ...   
9852      ZY                    Zymergen Inc.    2.43   
9853    ZYME                   Zymeworks Inc.    8.58   
9854    ZYNE    Zynerba Pharmaceuticals, Inc.  0.3359   
9855    ZYXI                      Zynex, Inc.    9.44   
9856     ZZZ                                     0.01   

                     exchange exchangeShortName   type  
0     New York Stock Exchange              NYSE  stock  
1     New York Stock Exchange              NYSE  stock  
2     New York Stock Exchange              NYSE  stock  
3     New York Stock Exchange              NYSE  stock  
4     New York Stock Exchange 

/Users/zhangwenbo/miniforge3/lib/python3.10/site-packages/cashare/minute_data.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lsss, ignore_index=True)
/Users/zhangwenbo/miniforge3/lib/python3.10/site-packages/cashare/minute_data.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lsss, ignore_index=True)


    stock_id                 date     open      low     high   close  volume
0          A  2023-06-29 09:30:00  116.160  116.090  116.320  116.22   15365
1          A  2023-06-29 09:31:00  116.230  116.230  116.370  116.35    3231
2          A  2023-06-29 09:32:00  116.410  116.080  116.460  116.08    4635
3          A  2023-06-29 09:33:00  116.080  116.070  116.300  116.23    1978
4          A  2023-06-29 09:34:00  116.340  116.165  116.350  116.24    3608
..       ...                  ...      ...      ...      ...     ...     ...
986        A  2023-07-03 12:58:00  119.245  119.245  119.315  119.27    8790
987        A  2023-07-03 12:59:00  119.280  119.220  119.400  119.24   42687
988        A  2023-07-03 13:00:00  119.320  119.320  119.320  119.32  164467
989        A  2023-07-03 13:10:00  119.320  119.320  119.320  119.32       0
990        A  2023-07-03 15:30:00  119.320  119.320  119.320  119.32       0

[991 rows x 7 columns]
(991, 7)
   percent name                     ts_code

/Users/zhangwenbo/miniforge3/lib/python3.10/site-packages/cashare/minute_data.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lsss, ignore_index=True)
/Users/zhangwenbo/miniforge3/lib/python3.10/site-packages/cashare/minute_data.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lsss, ignore_index=True)


    stock_id                 date    open     low     high    close  volume
0         AA  2023-06-29 09:30:00  33.500  33.370  33.5000  33.3914   68210
1         AA  2023-06-29 09:31:00  33.390  33.310  33.4200  33.3900   15672
2         AA  2023-06-29 09:32:00  33.400  33.280  33.4000  33.3000   18793
3         AA  2023-06-29 09:33:00  33.290  33.280  33.3600  33.3000   19072
4         AA  2023-06-29 09:34:00  33.278  33.210  33.3637  33.3637   18269
..       ...                  ...     ...     ...      ...      ...     ...
990       AA  2023-07-03 12:58:00  34.230  34.225  34.2800  34.2750   26599
991       AA  2023-07-03 12:59:00  34.270  34.270  34.3000  34.2800   80631
992       AA  2023-07-03 13:00:00  34.270  34.270  34.2700  34.2700  124243
993       AA  2023-07-03 13:10:00  34.270  34.270  34.2700  34.2700       0
994       AA  2023-07-03 15:30:00  34.270  34.270  34.2700  34.2700       0

[995 rows x 7 columns]
(995, 7)
   percent name            ts_code
0        0   AA  Alc

...

    stock_id                 date    open     low     high     close volume
0       AEMD  2023-06-29 09:30:00  0.4310  0.4050  0.43490  0.405000  59966
1       AEMD  2023-06-29 09:32:00  0.4176  0.4051  0.43009  0.405201  20117
2       AEMD  2023-06-29 09:34:00  0.4301  0.4301  0.43010  0.430100    390
3       AEMD  2023-06-29 09:35:00  0.4301  0.4289  0.43010  0.428900   1723
4       AEMD  2023-06-29 09:36:00  0.4289  0.4289  0.42890  0.428900    150
..       ...                  ...     ...     ...      ...       ...    ...
215     AEMD  2023-07-03 12:01:00  0.3500  0.3500  0.35010  0.350099   2424
216     AEMD  2023-07-03 12:04:00  0.3501  0.3501  0.35010  0.350100   3000
217     AEMD  2023-07-03 12:22:00  0.3501  0.3501  0.35010  0.350100    100
218     AEMD  2023-07-03 12:27:00  0.3500  0.3500  0.35000  0.350000    612
219     AEMD  2023-07-03 12:33:00  0.3500  0.3498  0.35000  0.349800   1350

[220 rows x 7 columns]
(220, 7)
   percent  name                ts_code
0        0  AEM

/Users/zhangwenbo/miniforge3/lib/python3.10/site-packages/cashare/minute_data.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lsss, ignore_index=True)
/Users/zhangwenbo/miniforge3/lib/python3.10/site-packages/cashare/minute_data.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lsss, ignore_index=True)


  stock_id                 date  open  low  high  close volume
0     AENZ  2023-06-30 09:30:00   1.9  1.9   1.9    1.9    103
(1, 7)


/Users/zhangwenbo/miniforge3/lib/python3.10/site-packages/cashare/minute_data.py:96: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lsss, ignore_index=True)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)